In [964]:
# IMPORT LIBRARY

import pandas as pd
import numpy as np
import re
import datetime

# `Read Excel File`

In [965]:
# read the Excel file on the specified sheet
df_input = pd.read_excel('CAOT Test Case _ Final.xlsx', sheet_name='Input Data - FBL5N')
df_hierarchy = pd.read_excel('CAOT Test Case _ Final.xlsx', sheet_name='Input Data - Cust Hierarchy')
df_holiday = pd.read_excel('CAOT Test Case _ Final.xlsx', sheet_name='Input - Holiday')
df_report = pd.read_excel('CAOT Test Case _ Final.xlsx', sheet_name='Input - Report Date')

# `VLOOKUP`

In [966]:
df_input.columns

Index(['Company Code', 'Account', 'Billing Document', 'Document Number',
       'Document type', 'Posting Key', 'Debit/Credit ind', 'Assignment',
       'Document Date', 'Special G/L ind.', 'Amount in local currency',
       'Local Currency', 'Amount in doc. curr.', 'Document currency',
       'Amount in loc.curr.2', 'Local Currency 2', 'G/L Account',
       'Posting Date', 'Baseline Payment Dte', 'Entry Date', 'Net due date',
       'Year/month', 'Reason code', 'User Name', 'Sales Document', 'Vendor',
       'Customer', 'Reference', 'Text', 'Clearing Document', 'Clearing date',
       'Arrears after net due date', 'Reference Key 1', 'Reference Key 2'],
      dtype='object')

In [967]:
# columns name on df_hierarchy
list_hierarchy_col = ['Customer Name', 'TFL2', 'Customer Channel', 'SWL2', 'SWL3', 'Banner']

# columns name which are insert to df_input
list_hierarchy_col_name = ['Customer Name', 'Trade Format Level 2', 'Channel', 'Sales Workforce Lv2', 'Sales Workforce Lv3', 'Banner Name']

In [968]:
for col_names in list_hierarchy_col:
    # Create list result from merge df_input and df_hierarchy
    list_name = [pd.merge(df_input, df_hierarchy, how='left', left_on='Account', right_on = 'Customer')[col_names]]

    # Insert list to df_input on specific column's number
    df_input.insert(list_hierarchy_col.index(col_names)+2, list_hierarchy_col_name[list_hierarchy_col.index(col_names)], list_name[0])

    # Replace if there missing value or not found in df_hierarchy
    df_input[list_hierarchy_col_name[list_hierarchy_col.index(col_names)]].fillna('Not Found', inplace=True)


# `LOGIC`

## `Exclude Column`

In [969]:
list_exlude = []
for row in range(len(df_input)):

    # Statement 1
    if df_input.loc[row,'Reference Key 1'] in (['CITDUMMYUFS', 'CITDUMMY UFS', 'CITDUMMY', 'CITDUMMY-NKW']):
        list_exlude.append(df_input.loc[row,'Reference Key 1'])

    # Statement 2
    elif df_input.loc[row,'Amount in local currency'] > 0:
        list_exlude.append('Positive Value')

    # Statement 3
    elif (df_input.loc[row,'Local Currency'] != 'IDR') | (df_input.loc[row,'Document currency'] != 'IDR'):
        list_exlude.append('Not IDR')

    # Statement 4
    elif str(df_input.loc[row, 'Document Number']).startswith('58'):
        list_exlude.append('Reversal document')

    # Statement 5
    elif (str(df_input['Reference'].iloc[row]).__contains__('bnppay')) | (str(df_input['Text'].iloc[row]).__contains__('bnppay')):
        list_exlude.append('BNNPAY')
    else:
        list_exlude.append(np.nan)

df_input['Exclude'] = list_exlude

In [970]:
# Check value 
df_input['Exclude'].value_counts()

Positive Value       1278
CITDUMMY              176
CITDUMMYUFS           105
Not IDR                 8
CITDUMMY-NKW            7
BNNPAY                  2
CITDUMMY UFS            2
Reversal document       2
Name: Exclude, dtype: int64

## `Days Column`

In [971]:
# Create variable represent report date from 'Input - Report Date' Sheet
report_date = df_report.columns[0].date()
report_date

datetime.date(2023, 2, 7)

In [972]:
# Create variable contains holiday from 'Input-Holiday' Sheet
holidays = df_holiday['Holiday'].dt.date.to_list()
holidays

[datetime.date(2023, 1, 1),
 datetime.date(2023, 2, 2),
 datetime.date(2023, 2, 28),
 datetime.date(2023, 3, 3),
 datetime.date(2023, 4, 15),
 datetime.date(2023, 5, 1),
 datetime.date(2023, 5, 2),
 datetime.date(2023, 5, 3),
 datetime.date(2023, 5, 16),
 datetime.date(2023, 5, 26),
 datetime.date(2023, 6, 1),
 datetime.date(2023, 7, 9),
 datetime.date(2023, 7, 30),
 datetime.date(2023, 8, 17),
 datetime.date(2023, 10, 8),
 datetime.date(2023, 12, 25)]

In [973]:
# Create dummy variable which represent changing missing value in 'Clearing Document' with 'none'
column_clear = df_input['Clearing Document'].fillna('none')

In [974]:
list_days = []

# Create workdays format, 1 equals to work, 0 equals to holiday(weekend)
workformat = [1,1,1,1,1,1,0]


for row in range(len(df_input)):

    # Statement if 'Clearing Date' is missing value
    if column_clear.iloc[row] == 'none':
        
        # counting workdays
        day_count = (np.busday_count(
            begindates = df_input[['Document Date']].iloc[row][0].date(), 
            enddates =report_date, 
            weekmask=workformat, 
            holidays= holidays))-1

        # Statement if day_count below than 0    
        if day_count < 0:
            list_days.append(0)
        else :
            list_days.append(day_count)

    # Statement if 'Clearing Date' has value
    else:

        # counting workdays
        day_count = (np.busday_count(
            begindates =df_input[['Document Date']].iloc[row][0].date(),
            enddates = df_input[['Clearing date']].iloc[row][0].date(), 
            weekmask=workformat, 
            holidays= holidays))-1

        # Statement if day_count below than 0    
        if day_count  < 0:
            list_days.append(0)

        else:
            list_days.append(day_count)

In [975]:
# Adding new column contains days
df_input['Days'] = list_days

In [976]:
# Check value 
df_input['Days'].value_counts()

0    2281
2     248
3     211
1      73
Name: Days, dtype: int64

## `Ageing Column`

In [977]:
list_ageing = []
for i in range(len(df_input)):

    # Statement based on number of Days column
    # Because there's no value < 0 (negative), then conditional statement "Days <= -1" will not incluiding
    if df_input['Days'].iloc[i] < 2:
        ageing = '<2'
    elif df_input['Days'].iloc[i] == 2:
        ageing = '=2'
    else:
        ageing = '>2'

    list_ageing.append(ageing)
    
    


In [978]:
# Check value
pd.DataFrame(list_ageing)[0].value_counts()

<2    2354
=2     248
>2     211
Name: 0, dtype: int64

In [979]:
# Adding new column which contains list_ageing
df_input['Ageing'] = list_ageing

## `Clearing Status`

In [980]:
list_status = []
for row in range(len(df_input)):

    # Statement based on 'Clearing Document' --> missing value has replaced with 'none' in column_clear
    if column_clear.iloc[row] == 'none':
        list_status.append('Open')
    else:
        list_status.append('Cleared')

In [981]:
# Check value
pd.DataFrame(list_status)[0].value_counts()

Cleared    2285
Open        528
Name: 0, dtype: int64

In [982]:
# Adding new column contain list_status
df_input['Clearing Status'] =list_status

# `GROUP OUTPUT`

## `Exclude field`

In [983]:
# Value counts including NaN and convert into DataFrame
exclude = pd.DataFrame(df_input['Exclude'].value_counts(dropna=False)).reset_index()\
    .rename(columns={'index': 'Row Labels', 'Exclude': 'Count of Billing Document'})
    
# Changing NaN value with '(blanks)' as written in 'Output' Sheet
exclude['Row Labels'] = exclude['Row Labels'].fillna('(blanks)')
exclude

,Row Labels,Count of Billing Document
0,Positive Value,1278
1,(blanks),1233
2,CITDUMMY,176
3,CITDUMMYUFS,105
4,Not IDR,8
5,CITDUMMY-NKW,7
6,BNNPAY,2
7,CITDUMMY UFS,2
8,Reversal document,2


In [984]:
# Create function to add total of numerical data type on last row index
def grand_total(dataframe):
    
    # sum of numerical data type
    dataframe.loc['Grand Total'] = dataframe.select_dtypes(include=['number']).sum()

    # Replace NaN from object data type to 'Grand Total'
    dataframe['Row Labels'] = dataframe['Row Labels'].fillna('Grand Total')

    dataframe_result = dataframe.reset_index(drop='True')
    
    return dataframe_result

In [985]:
exclude_final = grand_total(exclude)
exclude_final

,Row Labels,Count of Billing Document
0,Positive Value,1278.0
1,(blanks),1233.0
2,CITDUMMY,176.0
3,CITDUMMYUFS,105.0
4,Not IDR,8.0
5,CITDUMMY-NKW,7.0
6,BNNPAY,2.0
7,CITDUMMY UFS,2.0
8,Reversal document,2.0
9,Grand Total,2813.0


## `Channel field`

In [986]:
channel = pd.DataFrame(df_input[(df_input['Exclude'].isna()==True) & (df_input['Ageing'].isin(['<2', '=2']))][['Channel']]\
    .value_counts()).reset_index().rename(columns={'Channel':'Row Labels', 0:'Count of Billing Document'})
channel

,Row Labels,Count of Billing Document
0,DT,536
1,MT,324
2,IC,277
3,UFS,41
4,U-PRO,9
5,Others,1


In [987]:
channel_final = grand_total(channel)
channel_final

,Row Labels,Count of Billing Document
0,DT,536.0
1,MT,324.0
2,IC,277.0
3,UFS,41.0
4,U-PRO,9.0
5,Others,1.0
6,Grand Total,1188.0


# `Export to Excel file`

In [988]:
import xlsxwriter

In [989]:
writer = pd.ExcelWriter('CAOT Test_OUTPUT.xlsx', engine='xlsxwriter')

workbook  = writer.book

# 'Output Sheet'
df_input.to_excel(writer, sheet_name='Output', index=False, startrow=0, startcol=0)

output_sheet = writer.sheets['Output']

num_rows, num_cols = df_input.shape
output_sheet.autofilter(0, 0, num_rows, num_cols-1)


# 'Group by Output' sheet
exclude_final.to_excel(writer, sheet_name='Group by Output', index=False, startrow=4, startcol=0)
channel_final.to_excel(writer, sheet_name='Group by Output', index=False, startrow=5, startcol=4)

# write the first dataframe to the worksheet
group_sheet = writer.sheets['Group by Output']
group_sheet.write(0, 0, 'By Exclude field- Count of row')
group_sheet.write(0, 4, 'By Channel- Count of Days ≤ 2 days')

filter_1 = 'A5:A5'
group_sheet.autofilter(filter_1)

filter_2 = 'E6:E6'
group_sheet.autofilter(filter_2)


# save the Excel file
writer.save()